In [1]:
!pip install pandas
!pip install plotly

In [2]:
import pandas as pd

factors_path = "data\cancer_factors\eso_factors.csv"

factors = pd.read_csv(factors_path)
print(factors.head())

     State   County  Start Year  End Year  Value  StateFIPS  CountyFIPS  \
0  Alabama  Autauga        2015      2019   4.23        1.0      1001.0   
1  Alabama  Baldwin        2015      2019   5.09        1.0      1003.0   
2  Alabama  Barbour        2015      2019   4.04        1.0      1005.0   
3  Alabama     Bibb        2015      2019   3.71        1.0      1007.0   
4  Alabama   Blount        2015      2019   4.40        1.0      1009.0   

   drinking  obesity  diabetes  heart_disease  poverty  noHealthIns  smoking  
0     13.55    35.70     12.80           7.45    12.95         9.70    19.35  
1     15.75    29.85     11.95           7.65     9.95        12.05    18.20  
2     11.75    40.55     19.40          10.60    29.00        13.25    25.70  
3     14.35    40.10     14.15           8.40    21.05        10.80    23.80  
4     13.95    36.45     13.45           8.75    14.75        14.20    21.80  


In [3]:
# import plotly.express as px
# df = factors.melt(id_vars=["State", "County", "Start Year", "End Year", "StateFIPS", "CountyFIPS", "Value"],
#                   var_name="factor", value_name="factor_value")
# df.head()


In [4]:
# y_min, y_max = int(df["Value"].min()), int(df["Value"].max())
# x_min, x_max = int(df["factor_value"].min()), int(df["factor_value"].max())
# fig = px.scatter(df, x="factor_value", y="Value", animation_frame="factor", 
#                 range_x=[int(x_min), int(x_max)], range_y=[int(y_min), int(y_max)], 
#                 title="Cancer Incidence vs Various Factors", trendline="ols", 
#                 color="factor", trendline_scope="trace")

# fig["layout"].pop("updatemenus")
# fig.show()

In [5]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Load all cancer data files
cancer_files = {
    'Esophageal': 'data/cancer_factors/eso_factors.csv',
    'Kidney': 'data/cancer_factors/kidney_factors.csv', 
    'Liver': 'data/cancer_factors/liver_factors.csv',
    'Lung': 'data/cancer_factors/lung_factors.csv',
    'Pancreatic': 'data/cancer_factors/pancreatic_factors.csv',
    'Prostate': 'data/cancer_factors/prostate_factors.csv',
    'Skin': 'data/cancer_factors/skin_factors.csv'
}

# Load data
data = {}
for cancer_type, filename in cancer_files.items():
    data[cancer_type] = pd.read_csv(filename)

# Available factors
factors = ['drinking', 
           'obesity', 
           'diabetes', 
           'heart_disease', 
           'poverty', 
           'noHealthIns', 
           'smoking']

# Create interactive widgets
cancer_dropdown = widgets.Dropdown(
    options=list(cancer_files.keys()),
    value='Esophageal',
    description='Cancer Type:'
)

factor_dropdown = widgets.Dropdown(
    options=factors,
    value='smoking',
    description='Factor:'
)

# Plot function
def update_plot(cancer_type, factor):
    df = data[cancer_type]
    
    fig = px.scatter(
        df, 
        x=factor, 
        y='Value',
        title=f'{cancer_type} Cancer Rate vs {factor.title()}',
        labels={
            factor: factor.title().replace('_', ' '),
            'Value': f'{cancer_type} Cancer Rate'
        },
        hover_data=['State', 'County'],
        trendline='ols',
        trendline_scope='overall',
        color="State"
    )
    
    fig.update_layout(
        width=800,
        height=600,
        showlegend=False
    )
    
    fig.show()

# Interactive widget
interactive_plot = widgets.interactive(
    update_plot,
    cancer_type=cancer_dropdown,
    factor=factor_dropdown
)

display(interactive_plot)

interactive(children=(Dropdown(description='Cancer Type:', options=('Esophageal', 'Kidney', 'Liver', 'Lung', '…

In [6]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Load all cancer data files
cancer_files = {
    'Esophageal': 'data/cancer_factors/eso_factors.csv',
    'Kidney': 'data/cancer_factors/kidney_factors.csv',
    'Liver': 'data/cancer_factors/liver_factors.csv',
    'Lung': 'data/cancer_factors/lung_factors.csv',
    'Pancreatic': 'data/cancer_factors/pancreatic_factors.csv',
    'Prostate': 'data/cancer_factors/prostate_factors.csv',
    'Skin': 'data/cancer_factors/skin_factors.csv'
}

# Load data
data = {}
for cancer_type, filename in cancer_files.items():
    data[cancer_type] = pd.read_csv(filename)

# Available factors
factors = ['drinking',
           'obesity',
           'diabetes',
           'heart_disease',
           'poverty',
           'noHealthIns',
           'smoking']

# Create interactive widgets
cancer_dropdown = widgets.Dropdown(
    options=list(cancer_files.keys()),
    value='Esophageal',
    description='Cancer Type:'
)

factor_dropdown = widgets.Dropdown(
    options=factors,
    value='smoking',
    description='Factor:'
)

# Plot function for dot density maps
def update_dot_density_maps(cancer_type, factor):
    df = data[cancer_type]
    
    # Create subplot with 2 maps side by side
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[f'{factor.title().replace("_", " ")} Rate', f'{cancer_type} Cancer Rate'],
        specs=[[{"type": "geo"}, {"type": "geo"}]]
    )
    
    # Map 1: Factor rates
    fig.add_trace(
        px.choropleth(
            df,
            geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
            locations='CountyFIPS',
            color=factor,
            hover_data=['State', 'County'],
            color_continuous_scale="Viridis"
        ).data[0],
        row=1, col=1
    )
    
    # Map 2: Cancer rates
    fig.add_trace(
        px.choropleth(
            df,
            geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
            locations='CountyFIPS',
            color='Value',
            hover_data=['State', 'County'],
            color_continuous_scale="Viridis"
        ).data[0],
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title_text=f'{cancer_type} Cancer vs {factor.title().replace("_", " ")} - Dot Density Comparison',
        geo=dict(
            scope='usa',
            projection_type='albers usa',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            coastlinecolor='rgb(204, 204, 204)',
        ),
        geo2=dict(
            scope='usa',
            projection_type='albers usa',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            coastlinecolor='rgb(204, 204, 204)',
        ),
        height=600,
        width=1200,
        showlegend=False
    )
    
    fig.show()

# Interactive widget
interactive_maps = widgets.interactive(
    update_dot_density_maps,
    cancer_type=cancer_dropdown,
    factor=factor_dropdown
)

display(interactive_maps)

interactive(children=(Dropdown(description='Cancer Type:', options=('Esophageal', 'Kidney', 'Liver', 'Lung', '…